In [5]:
#### Notebook to munge LCD data - DEN
#### Written by: Jennifer.E.Kay@colorado.edu
#### Last update: March 11, 2022

In [6]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from metpy.units import units

from datetime import datetime

import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
import timeit
import scipy.stats as stats
import glob

In [7]:
df1 = pd.read_csv("DEN_2012-2021_2891846.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
df1

df0 = pd.read_csv("DEN_2002-2011_2891843.csv",usecols=[1,43,44,45,49,52,55,56,57],low_memory=False)
df0

##df0 = pd.read_csv("GrandJunction_2011-2012_2891886.csv",usecols=[1,43,44,45,49,52,55,56,57],dtype='str')

### could also try to Specify dtype option on import OR low_memory=False

,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2002-01-01T00:36:00,18,19,NaN,93,NaN,060,NaN,9
1,2002-01-01T00:45:00,19,19,T,100,NaN,060,NaN,10
2,2002-01-01T00:53:00,17,19,0.00,92,NaN,070,NaN,9
3,2002-01-01T01:20:00,18,18,T,100,NaN,060,NaN,8
4,2002-01-01T01:44:00,18,18,NaN,100,NaN,060,NaN,6
...,...,...,...,...,...,...,...,...,...
132534,2011-12-31T22:53:00,10,20,0.00,65,24.82,160,NaN,13
132535,2011-12-31T23:00:00,10,20,NaN,65,24.87,160,NaN,13
132536,2011-12-31T23:53:00,11,23,0.00,60,24.82,160,NaN,13
132537,2011-12-31T23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
included_cols = ['DATE','HourlyDryBulbTemperature',
                 'HourlyDewPointTemperature','HourlyPrecipitation',
                'HourlyRelativeHumidity',
                'HourlyStationPressure','HourlyWindDirection',
                'HourlyWindGustSpeed','HourlyWindSpeed'] # hourly fields only
data0 = df0.loc[:, df0.columns.isin(list(included_cols))]
data1 = df1.loc[:, df1.columns.isin(list(included_cols))]
df = pd.concat([data0,data1])
print(df.shape)
df

(394744, 9)


,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2002-01-01T00:36:00,18,19,NaN,93,NaN,060,NaN,9
1,2002-01-01T00:45:00,19,19,T,100,NaN,060,NaN,10
2,2002-01-01T00:53:00,17,19,0.00,92,NaN,070,NaN,9
3,2002-01-01T01:20:00,18,18,T,100,NaN,060,NaN,8
4,2002-01-01T01:44:00,18,18,NaN,100,NaN,060,NaN,6
...,...,...,...,...,...,...,...,...,...
262200,2021-12-31T22:55:00,14,16,NaN,93.0,NaN,040,NaN,3
262201,2021-12-31T23:15:00,14,16,NaN,93.0,NaN,030,NaN,5
262202,2021-12-31T23:35:00,14,16,NaN,93.0,NaN,030,NaN,6
262203,2021-12-31T23:55:00,12,14,NaN,92.0,24.43,020,NaN,6


In [9]:
### Print the Shape of the data frame

print(df.shape)

(394744, 9)


In [10]:
#df.to_csv('72476023066_GrandJunction_2002-2021_hourlydata.csv', index=False)
### Read in your data
#df = pd.read_csv("72476023066_GrandJunction_2002-2021_hourlydata.csv",low_memory=False)
### Preview the values in the data frame by printing it
#print(df)

In [11]:
## make the datetime format is a readable datetime

##create a datetime object from a string
#foo = datetime.strptime('2012-01-01', '%Y-%m-%d')
#foo = datetime.strptime('2012-01-01T00:53:00', '%Y-%m-%dT%H:%M:%S')
#foo = datetime.strptime(df['DATE'][0], '%Y-%m-%dT%H:%M:%S')
#print(foo)

df['datetime'] = pd.to_datetime(df['DATE'], format='%Y-%m-%dT%H:%M:%S')
#df

In [12]:
## change the order of the data frame variables
df = df[['DATE','datetime','HourlyDryBulbTemperature','HourlyDewPointTemperature','HourlyPrecipitation',
                'HourlyRelativeHumidity','HourlyStationPressure','HourlyWindDirection','HourlyWindSpeed','HourlyWindGustSpeed']]
df

,DATE,datetime,HourlyDryBulbTemperature,HourlyDewPointTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyWindDirection,HourlyWindSpeed,HourlyWindGustSpeed
0,2002-01-01T00:36:00,2002-01-01 00:36:00,19,18,NaN,93,NaN,060,9,NaN
1,2002-01-01T00:45:00,2002-01-01 00:45:00,19,19,T,100,NaN,060,10,NaN
2,2002-01-01T00:53:00,2002-01-01 00:53:00,19,17,0.00,92,NaN,070,9,NaN
3,2002-01-01T01:20:00,2002-01-01 01:20:00,18,18,T,100,NaN,060,8,NaN
4,2002-01-01T01:44:00,2002-01-01 01:44:00,18,18,NaN,100,NaN,060,6,NaN
...,...,...,...,...,...,...,...,...,...,...
262200,2021-12-31T22:55:00,2021-12-31 22:55:00,16,14,NaN,93.0,NaN,040,3,NaN
262201,2021-12-31T23:15:00,2021-12-31 23:15:00,16,14,NaN,93.0,NaN,030,5,NaN
262202,2021-12-31T23:35:00,2021-12-31 23:35:00,16,14,NaN,93.0,NaN,030,6,NaN
262203,2021-12-31T23:55:00,2021-12-31 23:55:00,14,12,NaN,92.0,24.43,020,6,NaN


In [13]:
## rename the columns, shorter and include units
df.rename(columns={'HourlyDryBulbTemperature': 'drytemp_F', 'HourlyDewPointTemperature': 'dewtemp_F',
                   'HourlyPrecipitation': 'prec_inch', 'HourlyRelativeHumidity': 'RH',
                   'HourlyStationPressure': 'pres_Hg', 'HourlyWindDirection': 'wdir',
                   'HourlyWindSpeed': 'wspd_mph', 'HourlyWindGustSpeed': 'wgust_mph'}, inplace=True)
df.iloc[0]

DATE         2002-01-01T00:36:00
datetime     2002-01-01 00:36:00
drytemp_F                     19
dewtemp_F                     18
prec_inch                    NaN
RH                            93
pres_Hg                      NaN
wdir                         060
wspd_mph                       9
wgust_mph                    NaN
Name: 0, dtype: object

In [14]:
## convert pressure from a string to a float
df["pres_Hg"] = pd.to_numeric(df["pres_Hg"], downcast="float",errors='coerce')
## Deal with the mix of text and numeric values!  Set text values to NaN using coerce!!

## convert units of pressure from inches Mercury to mbars
#print(df['pres_Hg'][1])
#print((df['pres_Hg'][1]* units.inHg).to(units.mbar))
## example converting to mbar from inches of mercury
print((25.53 * units.inHg).to(units.mbar))

df['pres_Hg']=(df.pres_Hg.values * units('inHg')).to('mbar')

df.rename(columns={'pres_Hg': 'pres_mbar'}, inplace=True)


864.5450198790573 millibar


In [15]:
df["drytemp_F"] = pd.to_numeric(df["drytemp_F"], downcast="float",errors='coerce')
df["dewtemp_F"] = pd.to_numeric(df["dewtemp_F"], downcast="float",errors='coerce')
df["prec_inch"] = pd.to_numeric(df["prec_inch"], downcast="float",errors='coerce')
df["RH"] = pd.to_numeric(df["RH"], downcast="float",errors='coerce')
df["wdir"] = pd.to_numeric(df["wdir"], downcast="float",errors='coerce')
df["wspd_mph"] = pd.to_numeric(df["wspd_mph"], downcast="float",errors='coerce')
df["wgust_mph"] = pd.to_numeric(df["wgust_mph"], downcast="float",errors='coerce')
df

,DATE,datetime,drytemp_F,dewtemp_F,prec_inch,RH,pres_mbar,wdir,wspd_mph,wgust_mph
0,2002-01-01T00:36:00,2002-01-01 00:36:00,19.0,18.0,NaN,93.0,NaN,60.0,9.0,NaN
1,2002-01-01T00:45:00,2002-01-01 00:45:00,19.0,19.0,NaN,100.0,NaN,60.0,10.0,NaN
2,2002-01-01T00:53:00,2002-01-01 00:53:00,19.0,17.0,0.0,92.0,NaN,70.0,9.0,NaN
3,2002-01-01T01:20:00,2002-01-01 01:20:00,18.0,18.0,NaN,100.0,NaN,60.0,8.0,NaN
4,2002-01-01T01:44:00,2002-01-01 01:44:00,18.0,18.0,NaN,100.0,NaN,60.0,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...
262200,2021-12-31T22:55:00,2021-12-31 22:55:00,16.0,14.0,NaN,93.0,NaN,40.0,3.0,NaN
262201,2021-12-31T23:15:00,2021-12-31 23:15:00,16.0,14.0,NaN,93.0,NaN,30.0,5.0,NaN
262202,2021-12-31T23:35:00,2021-12-31 23:35:00,16.0,14.0,NaN,93.0,NaN,30.0,6.0,NaN
262203,2021-12-31T23:55:00,2021-12-31 23:55:00,14.0,12.0,NaN,92.0,827.2948,20.0,6.0,NaN


In [16]:
### deal with missing data

## copy the data array into a new array without missing values called (data_nomissing)
df_nomissing=df.copy()
df_nomissing
print(df_nomissing.shape)

option_missing='option2'

##### OPTION #1 = replace the NaN data with the mean value over the entire dataset.  Simple!
if option_missing=='option1':
    df_nomissing['drytemp_F']=df_nomissing['drytemp_F'].fillna(df['drytemp_F'].mean())
    df_nomissing['dewtemp_F']=df_nomissing['dewtemp_F'].fillna(df['dewtemp_F'].mean())
    df_nomissing['prec_inch']=df_nomissing['prec_inch'].fillna(0)
    df_nomissing['RH']=df_nomissing['RH'].fillna(df['RH'].mean())
    df_nomissing['pres_mbar']=df_nomissing['pres_mbar'].fillna(df['pres_mbar'].mean())
    df_nomissing['wdir']=df_nomissing['wdir'].fillna(df['wdir'].mean())
    df_nomissing['wspd_mph']=df_nomissing['wspd_mph'].fillna(df['wspd_mph'].mean())
    df_nomissing['wgust_mph']=df_nomissing['wgust_mph'].fillna(df['wgust_mph'].mean())
    
##### OPTION #2 = ffill: propagate last valid observation forward to next valid observation,
#### then backfill / bfill: use next valid observation to fill gap.
if option_missing=='option2':
     df_nomissing=df_nomissing.fillna(method='bfill')
     df_nomissing=df_nomissing.fillna(method='ffill')
    
df_nomissing


(394744, 10)


,DATE,datetime,drytemp_F,dewtemp_F,prec_inch,RH,pres_mbar,wdir,wspd_mph,wgust_mph
0,2002-01-01T00:36:00,2002-01-01 00:36:00,19.0,18.0,0.00,93.0,839.147156,60.0,9.0,26.0
1,2002-01-01T00:45:00,2002-01-01 00:45:00,19.0,19.0,0.00,100.0,839.147156,60.0,10.0,26.0
2,2002-01-01T00:53:00,2002-01-01 00:53:00,19.0,17.0,0.00,92.0,839.147156,70.0,9.0,26.0
3,2002-01-01T01:20:00,2002-01-01 01:20:00,18.0,18.0,0.00,100.0,839.147156,60.0,8.0,26.0
4,2002-01-01T01:44:00,2002-01-01 01:44:00,18.0,18.0,0.00,100.0,839.147156,60.0,6.0,26.0
...,...,...,...,...,...,...,...,...,...,...
262200,2021-12-31T22:55:00,2021-12-31 22:55:00,16.0,14.0,0.02,93.0,827.294800,40.0,3.0,18.0
262201,2021-12-31T23:15:00,2021-12-31 23:15:00,16.0,14.0,0.02,93.0,827.294800,30.0,5.0,18.0
262202,2021-12-31T23:35:00,2021-12-31 23:35:00,16.0,14.0,0.02,93.0,827.294800,30.0,6.0,18.0
262203,2021-12-31T23:55:00,2021-12-31 23:55:00,14.0,12.0,0.02,92.0,827.294800,20.0,6.0,18.0


In [17]:
df_nomissing.to_csv('72565003017_DEN_2002-2021_hourlydata_munged.csv', index=False)

In [ ]:
#df.to_csv('goo.csv', index=False)